# Solution of Lahti *et al.* 2014

### Write a function that takes as input a dictionary of constraints (i.e., selecting a specific group of records) and returns a dictionary tabulating the BMI group for all the records matching the constraints. For example, calling: 
```
get_BMI_count({'Age': '28', 'Sex': 'female'})
``` 
### should return:
```
{'NA': 3, 'lean': 8, 'overweight': 2, 'underweight': 1}
```

In [3]:
import csv # Import csv modulce for reading the file

We start my reading the Metadata file, set up a csv reader, and print the header and first few lines to get a feel for data.

In [5]:
with open('../data/Lahti2014/Metadata.tab') as f:
        csvr = csv.DictReader(f, delimiter = '\t')
        header = csvr.fieldnames
        print(header)
        # For each row
        for i, row in enumerate(csvr):
            print(row)
            if i > 2:
                break

['SampleID', 'Age', 'Sex', 'Nationality', 'DNA_extraction_method', 'ProjectID', 'Diversity', 'BMI_group', 'SubjectID', 'Time']
{'Age': '28', 'Diversity': '5.76', 'Sex': 'male', 'DNA_extraction_method': 'NA', 'BMI_group': 'severeobese', 'SubjectID': '1', 'Nationality': 'US', 'Time': '0', 'ProjectID': '1', 'SampleID': 'Sample-1'}
{'Age': '24', 'Diversity': '6.06', 'Sex': 'female', 'DNA_extraction_method': 'NA', 'BMI_group': 'obese', 'SubjectID': '2', 'Nationality': 'US', 'Time': '0', 'ProjectID': '1', 'SampleID': 'Sample-2'}
{'Age': '52', 'Diversity': '5.5', 'Sex': 'male', 'DNA_extraction_method': 'NA', 'BMI_group': 'lean', 'SubjectID': '3', 'Nationality': 'US', 'Time': '0', 'ProjectID': '1', 'SampleID': 'Sample-3'}
{'Age': '22', 'Diversity': '5.87', 'Sex': 'female', 'DNA_extraction_method': 'NA', 'BMI_group': 'underweight', 'SubjectID': '4', 'Nationality': 'US', 'Time': '0', 'ProjectID': '1', 'SampleID': 'Sample-4'}


It's time to decide on a data structure to record our result: For each row in the file, we want to make sure all the constraints are matching the desired ones. If so, we keep count of the BMI group. A dictionary with the BMI_groups as keys and counts as values will work well:

In [6]:
# Initiate an empty dictionary to keep track of counts per BMI_group
BMI_count = {}

In [10]:
# set up our dictionary of constraints for testing purposes
dict_constraints = {'Age': '28', 'Sex': 'female'}

OK, now the tricky part: for each row, we want to test if the constraints (a dictionary) matches the data (which itself is a dictionary). We can do it element-wise, that means we take a key from the data dictionary (`row`) and test if its value is NOT identical to the corresponding value in the constraint dictionary. We start out by setting the value `matching` to `TRUE` and set it to `FALSE` if we encounter a discripancy. This way, we stop immediately if one of the elements does not match and move on to the next row of data.

In [30]:
with open('../data/Lahti2014/Metadata.tab') as f:
    csvr = csv.DictReader(f, delimiter = '\t')
    for i, row in enumerate(csvr):        
        # check that all conditions are met
        matching = True
        for e in dict_constraints:
            if row[e] != dict_constraints[e]:
                # The constraint is not met. Move to the next record
                matching = False
                break
            print("in row", i, "the key", e,"in data does not match", e, "in constraints")
        if i > 5:
            break 

in row 0 the key Age in data does not match Age in constraints


In some rows, all constraints will be fulfillled (i.e., our `matching` variable will still be `TRUE` after checking all elements). In this case, we want to increase the count of that particular BMI_group in our result dictionary `BMI_count`. We can directly add one to the appropriate `BMI_group` if we have seen it before, else we initiate that key with a value of one:

In [16]:
with open('../data/Lahti2014/Metadata.tab') as f:
        csvr = csv.DictReader(f, delimiter = '\t')
        for row in csvr:
            # check that all conditions are met
            matching = True
            for e in dict_constraints:
                if row[e] != dict_constraints[e]:
                    # The constraint is not met. Move to the next record
                    matching = False
                    break
            # matching is True only if all the constraints have been met
            if matching == True:
                # extract the BMI_group
                my_BMI = row['BMI_group']
                if my_BMI in BMI_count.keys():
                    # If we've seen it before, add one record to the count
                    BMI_count[my_BMI] = BMI_count[my_BMI] + 1
                else:
                    # If not, initialize at 1
                    BMI_count[my_BMI] = 1

In [17]:
BMI_count

{'NA': 3, 'lean': 8, 'overweight': 2, 'underweight': 1}

Excellent! Now, we can put everything together and create a function that accepts our constraint dictionary. Remember to document everything nicely:

In [5]:
def get_BMI_count(dict_constraints):
    """ Take as input a dictionary of constraints
        for example, {'Age': '28', 'Sex': 'female'}
        And return the count of the various groups of BMI
    """
    # We use a dictionary to store the results
    BMI_count = {}
    # Open the file, build a csv DictReader
    with open('../data/Lahti2014/Metadata.tab') as f:
        csvr = csv.DictReader(f, delimiter = '\t')
        # For each row
        for row in csvr:
            # check that all conditions are met
            matching = True
            for e in dict_constraints:
                if row[e] != dict_constraints[e]:
                    # The constraint is not met. Move to the next record
                    matching = False
                    break
            # matching is True only if all the constraints have been met
            if matching == True:
                # extract the BMI_group
                my_BMI = row['BMI_group']
                if my_BMI in BMI_count.keys():
                    # If we've seen it before, add one record to the count
                    BMI_count[my_BMI] = BMI_count[my_BMI] + 1
                else:
                    # If not, initialize at 1
                    BMI_count[my_BMI] = 1
    return BMI_count

In [6]:
get_BMI_count({'Nationality': 'US', 'Sex': 'female'})

{'lean': 12, 'obese': 3, 'overweight': 5, 'severeobese': 1, 'underweight': 3}

### Write a function that takes as input the constraints (as above), and a bacterial "genus". The function returns the average abundance (in logarithm base 10) of the genus for each group of BMI in the sub-population. For example, calling:
```
get_abundance_by_BMI({'Time': '0', 'Nationality': 'US'}, 'Clostridium difficile et rel.')
```
### should return:
```
____________________________________________________________________
Abundance of Clostridium difficile et rel. In sub-population:
____________________________________________________________________
Nationality -> US
Time -> 0
____________________________________________________________________
3.08 	 NA
3.31 	 underweight
3.84 	 lean
2.89 	 overweight
3.31 	 obese
3.45 	 severeobese
____________________________________________________________________
```

To solve this task, we can recycle quite a bit of code that we just developed. However, instead of just counting occurances of `BMI_group`s, we want to keep track of the records (i.e., `SampleID`s) that match our constraints and look up a specific bacteria abundance in the file `HITChip.tab`. 
First, we create a dictionary with all records that match our constraints:

In [31]:
# We use a dictionary to store the results
BMI_IDs = {}
# Open the file, build a csv DictReader
with open('../data/Lahti2014/Metadata.tab') as f:
    csvr = csv.DictReader(f, delimiter = '\t')
    for row in csvr:
        # check that all conditions are met
        matching = True
        for e in dict_constraints:
            if row[e] != dict_constraints[e]:
                # The constraint is not met. Move to the next record
                matching = False
                break
        # matching is True only if all the constraints have been met
        if matching == True:
            # extract the BMI_group
            my_BMI = row['BMI_group']
            if my_BMI in BMI_IDs.keys():
                # If we've seen it before, add the SampleID
                BMI_IDs[my_BMI] = BMI_IDs[my_BMI] + [row['SampleID']]
            else:
                # If not, initialize
                BMI_IDs[my_BMI] = [row['SampleID']]

In [32]:
BMI_IDs

{'NA': ['Sample-889', 'Sample-892', 'Sample-893'],
 'lean': ['Sample-17',
  'Sample-22',
  'Sample-153',
  'Sample-208',
  'Sample-753',
  'Sample-754',
  'Sample-785',
  'Sample-963'],
 'overweight': ['Sample-874', 'Sample-964'],
 'underweight': ['Sample-93']}

Before moving on, let's have a look at the `HITChip` file:

In [23]:
with open("../data/Lahti2014/HITChip.tab", "r") as HIT:
    csvr = csv.DictReader(HIT, delimiter = "\t")
    header = csvr.fieldnames
    print(header)
    for i, row in enumerate(csvr):
        print(row)
        if i > 2:
            break

['SampleID', 'Actinomycetaceae', 'Aerococcus', 'Aeromonas', 'Akkermansia', 'Alcaligenes faecalis et rel.', 'Allistipes et rel.', 'Anaerobiospirillum', 'Anaerofustis', 'Anaerostipes caccae et rel.', 'Anaerotruncus colihominis et rel.', 'Anaerovorax odorimutans et rel.', 'Aneurinibacillus', 'Aquabacterium', 'Asteroleplasma et rel.', 'Atopobium', 'Bacillus', 'Bacteroides fragilis et rel.', 'Bacteroides intestinalis et rel.', 'Bacteroides ovatus et rel.', 'Bacteroides plebeius et rel.', 'Bacteroides splachnicus et rel.', 'Bacteroides stercoris et rel.', 'Bacteroides uniformis et rel.', 'Bacteroides vulgatus et rel.', 'Bifidobacterium', 'Bilophila et rel.', 'Brachyspira', 'Bryantella formatexigens et rel.', 'Bulleidia moorei et rel.', 'Burkholderia', 'Butyrivibrio crossotus et rel.', 'Campylobacter', 'Catenibacterium mitsuokai et rel.', 'Clostridium (sensu stricto)', 'Clostridium cellulosi et rel.', 'Clostridium colinum et rel.', 'Clostridium difficile et rel.', 'Clostridium felsineum et re

We see that that each row contains the `SampleID` and abundance data for various phylogenetically clustered bacteria. For each row in the file, we can now check if we are interested in that particular `SampleID` (i.e., if it matched our constraint and is in our `BMI_IDs` dictionary). If so, we retrieve the abundance of the bacteria of interest and add it to the the previously identified abundances within a particular BMI_group. If we had not encounter this BMI_group before, we initiate the key with the abundance as value. As we want to calculate the mean of these abuncandes later, we also keep track of the number of occurances:

In [27]:
# set up dictionary to track abundances by BMI_group and number of identified records
abundance = {}
# choose a bacteria genus for testing
genus = "Clostridium difficile et rel."
with open('../data/Lahti2014/HITChip.tab') as f:
    csvr = csv.DictReader(f, delimiter = '\t')
    # For each row
    for row in csvr:
        # check whether we need this SampleID
        matching = False
        for g in BMI_IDs:
            if row['SampleID'] in BMI_IDs[g]:
                if g in abundance.keys():
                    abundance[g][0] = abundance[g][0] + float(row[genus])
                    abundance[g][1] = abundance[g][1] + 1

                else:
                    abundance[g] = [float(row[genus]), 1]
                # we have found it, so move on
                break

In [28]:
abundance

{'NA': [26653.01492637446, 3],
 'lean': [70636.33360838753, 8],
 'overweight': [3571.2360766130105, 2],
 'underweight': [3056.63707319002, 1]}

Now we take care of calculating the mean and printing the results. We need to load the scipy (or numby) module in order to calculate `log10`:

In [29]:
import scipy

print("____________________________________________________________________")
print("Abundance of " + genus + " In sub-population:")
print("____________________________________________________________________")
for key, value in dict_constraints.items():
    print(key, "->", value)
print("____________________________________________________________________")
for ab in ['NA', 'underweight', 'lean', 'overweight', 
           'obese', 'severeobese', 'morbidobese']:
    if ab in abundance.keys():
        abundance[ab][0] = scipy.log10(abundance[ab][0] / abundance[ab][1])
        print(round(abundance[ab][0], 2), '\t', ab)
print("____________________________________________________________________")
print("")

____________________________________________________________________
Abundance of Clostridium difficile et rel. In sub-population:
____________________________________________________________________
Age -> 28
Sex -> female
____________________________________________________________________
3.95 	 NA
3.49 	 underweight
3.95 	 lean
3.25 	 overweight
____________________________________________________________________



Last but not least, we put it all together in a function:

In [ ]:
import scipy # For log10

def get_abundance_by_BMI(dict_constraints, genus = 'Aerococcus'):
    # We use a dictionary to store the results
    BMI_IDs = {}
    # Open the file, build a csv DictReader
    with open('../data/Lahti2014/Metadata.tab') as f:
        csvr = csv.DictReader(f, delimiter = '\t')
        # For each row
        for row in csvr:
            # check that all conditions are met
            matching = True
            for e in dict_constraints:
                if row[e] != dict_constraints[e]:
                    # The constraint is not met. Move to the next record
                    matching = False
                    break
            # matching is True only if all the constraints have been met
            if matching == True:
                # extract the BMI_group
                my_BMI = row['BMI_group']
                if my_BMI in BMI_IDs.keys():
                    # If we've seen it before, add the SampleID
                    BMI_IDs[my_BMI] = BMI_IDs[my_BMI] + [row['SampleID']]
                else:
                    # If not, initialize
                    BMI_IDs[my_BMI] = [row['SampleID']]
    # Now let's open the other file, and keep track of the abundance of the genus for each 
    # BMI group
    abundance = {}
    with open('../data/Lahti2014/HITChip.tab') as f:
        csvr = csv.DictReader(f, delimiter = '\t')
        # For each row
        for row in csvr:
            # check whether we need this SampleID
            matching = False
            for g in BMI_IDs:
                if row['SampleID'] in BMI_IDs[g]:
                    if g in abundance.keys():
                        abundance[g][0] = abundance[g][0] + float(row[genus])
                        abundance[g][1] = abundance[g][1] + 1
                        
                    else:
                        abundance[g] = [float(row[genus]), 1]
                    # we have found it, so move on
                    break
    # Finally, calculate means, and print results
    print("____________________________________________________________________")
    print("Abundance of " + genus + " In sub-population:")
    print("____________________________________________________________________")
    for key, value in dict_constraints.items():
        print(key, "->", value)
    print("____________________________________________________________________")
    for ab in ['NA', 'underweight', 'lean', 'overweight', 
               'obese', 'severeobese', 'morbidobese']:
        if ab in abundance.keys():
            abundance[ab][0] = scipy.log10(abundance[ab][0] / abundance[ab][1])
            print(round(abundance[ab][0], 2), '\t', ab)
    print("____________________________________________________________________")
    print("")

In [8]:
get_abundance_by_BMI({'Time': '0', 'Nationality': 'US'}, 
                     'Clostridium difficile et rel.')

____________________________________________________________________
Abundance of Clostridium difficile et rel. In sub-population:
____________________________________________________________________
Nationality -> US
Time -> 0
____________________________________________________________________
3.08 	 NA
3.31 	 underweight
3.84 	 lean
2.89 	 overweight
3.31 	 obese
3.45 	 severeobese
____________________________________________________________________



### Repeat this analysis for all genera, and for the records having `Time = 0`.

A function to extract all the genera in the database:

In [10]:
def get_all_genera():
    with open('../data/Lahti2014/HITChip.tab') as f:
        header = f.readline().strip()
    genera = header.split('\t')[1:]
    return genera

Testing:

In [7]:
get_all_genera()[:6]

['Actinomycetaceae',
 'Aerococcus',
 'Aeromonas',
 'Akkermansia',
 'Alcaligenes faecalis et rel.',
 'Allistipes et rel.']

Now use this function to print the results for all genera at `Time = 0`:

In [8]:
for g in get_all_genera()[:5]:
    get_abundance_by_BMI({'Time': '0'}, g)

____________________________________________________________________
Abundance of Actinomycetaceae In sub-population:
____________________________________________________________________
Time -> 0
____________________________________________________________________
1.98 	 NA
1.95 	 underweight
1.98 	 lean
1.97 	 overweight
1.93 	 obese
1.95 	 severeobese
1.9 	 morbidobese
____________________________________________________________________

____________________________________________________________________
Abundance of Aerococcus In sub-population:
____________________________________________________________________
Time -> 0
____________________________________________________________________
1.66 	 NA
1.63 	 underweight
1.66 	 lean
1.66 	 overweight
1.61 	 obese
1.62 	 severeobese
1.6 	 morbidobese
____________________________________________________________________

____________________________________________________________________
Abundance of Aeromonas In sub-population:
_____